In [1]:
df = spark.read.csv('/FileStore/tables/tweets_cleaned.csv', header='true', inferSchema='true')

In [2]:
df.show()

In [3]:
df.printSchema()

In [4]:
df.select('tweet_text').show(truncate=False)

In [5]:
from pyspark.ml.feature import StopWordsRemover, Tokenizer

tokenizer = Tokenizer(inputCol='tweet_text', outputCol='words')

tweets = tokenizer.transform(df)

tweets.select('tweet_text', 'words').show(truncate=False)

In [6]:
stop_words_remover = StopWordsRemover(inputCol='words', outputCol='filtered')

tweets_filtered = stop_words_remover.transform(tweets)

tweets_filtered.select('words', 'filtered').show()

In [7]:
def remove_duplicates(l):
  return list(set(l))

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import *

removeUDF = udf(remove_duplicates, ArrayType(StringType()))

tDF = tweets_filtered.withColumn('filtered1', removeUDF('filtered'))

tDF.select('filtered', 'filtered1').show()

In [8]:
from pyspark.ml.fpm import FPGrowth, FPGrowthModel

fpgrowth = FPGrowth(itemsCol='filtered1', minSupport=0.2, minConfidence=0.2)
model = fpgrowth.fit(tDF)
model.freqItemsets.show()
model.associationRules.show()

In [9]:
model.transform(tDF).select('prediction').show()